In [1]:
import tensorflow
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, BatchNormalization, Flatten, Dense

import PIL
from PIL import Image

import sklearn
import pandas
import numpy
import json
import os
from sklearn.model_selection import train_test_split

In [2]:
%load_ext tensorboard

In [3]:
image_width = 150
image_height = 150
image_channels = 3
batch_size = 15
epochs = 1

In [4]:
model = tensorflow.keras.Sequential()

In [5]:
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape = (image_width, image_height, image_channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

In [6]:
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

In [7]:
model.add(Conv2D(150, (3, 3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

In [8]:
model.add(Flatten())

In [9]:
model.add(Dense(512, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [10]:
model.add(Dense(2, activation = 'softmax'))

In [11]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])

In [12]:
#model.summary()

In [13]:
types = ['jpg', 'png']
categories = []

cats = os.listdir("./data/train/cats")
dogs = os.listdir("./data/train/dogs")

cats = [cat for cat in cats if len(cat.split(".")) > 2 if cat.split(".")[2] in types]
dogs = [dog for dog in dogs if len(dog.split(".")) > 2 if dog.split(".")[2] in types]

for i in range(0, len(cats)):
    categories.append(0)

for i in range(0, len(dogs)):
    categories.append(1)
        
dataframe = pandas.DataFrame({
    'filename' : cats + dogs,
    'category' : categories
})

dataframe["category"] = dataframe["category"].replace({0: 'cat', 1: 'dog'})


In [14]:
class CustomCallback(tensorflow.keras.callbacks.Callback):
    
    def on_train_begin(self, logs = None):
        print("Starting training on this.date")
        
    def on_train_end(self, logs = None):
        print("Stopped training")
        
    def on_epoch_begin(self, epoch, logs = None):
        print("Epoch begin")
        
    def on_epoch_end(self, epoch, logs = None):
        print("Epoch end")
    
    def on_test_begin(self, logs = None):
        print("Test start")
        
    def on_test_end(self, logs = None):
        print("Test end")
        
    def on_predict_begin(self, logs = None):
        print("Predicting")
        
    def on_predict_end(self, logs = None):
        print("End predicting")
        
    
        
    


In [15]:
train_df, validate_df = train_test_split(dataframe, test_size = 0.20, random_state = 42)
train_df = train_df.reset_index(drop = True)
validate_df = validate_df.reset_index(drop = True)
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]

In [16]:
train_data_generator = tensorflow.keras.preprocessing.image.ImageDataGenerator(
   rotation_range = 15, 
   rescale = 1. / 255, 
   shear_range = 0.1, 
   zoom_range = 0.2, 
   horizontal_flip = True, 
   width_shift_range = 0.1, 
   height_shift_range = 0.1)

train_generator = train_data_generator.flow_from_directory("./data/train/", target_size = (image_width, image_height), class_mode = 'categorical', batch_size = batch_size)

validation_data_generator = tensorflow.keras.preprocessing.image.ImageDataGenerator(
   rescale = 1.0 / 255)

validation_generator = validation_data_generator.flow_from_directory("./data/validation/", target_size = (image_width, image_height), class_mode = 'categorical', batch_size = batch_size)

#model.save('catsndogs')

#validation_datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0 / 255)
#validation_generator = validation_datagen.flow_from_dataframe(validate_df, "./data/validation/",  x_col='filename', y_col='category', target_size = (image_width, image_height), class_mode = 'categorical', batch_size = batch_size)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [17]:
model.fit(train_generator, epochs = epochs, validation_data = validation_generator, validation_steps = 800, steps_per_epoch = 10)
model.save('test')

Epoch 1/5
 1/10 [==>...........................] - ETA: 0s - loss: 1.9609 - accuracy: 0.3333

KeyboardInterrupt: 